## Capstone Project - Battle of the Neighborhoods
Filipino Restaurant Analysis in Los Angeles, California

## Data

<b>Based on the criteria to identify the best areas in LA for our audience, the following factors that will influence the final decision are:</b>

- Number of existing filipino restaurants
- Number of and distance to filipino restaurants in the neighborhood 
- Distance of neighborhood to city's center
- Number of hotels in neighborhood
- Distance of neighborhood to nearby hotels



<b>The following data sources will be needed to extract/generate the required information: </b>
- List of all districts/neighborhods in LA https://en.wikipedia.org/wiki/List_of_districts_and_neighborhoods_of_Los_Angeles

- Coordinates of all neighbourhoods and venues — GeoPy Nominatim geocoding
- Number of filipino restaurants and their location in every neighbourhood — Foursquare API
- Number of hotels and their location in every neighborhod - Foursquare API

 Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
#Command to install OpenCage Geocoder for fetching Lat and Lng of Neighborhood
!pip install opencage

#Importing OpenCage Geocoder
from opencage.geocoder import OpenCageGeocode

# use the inline backend to generate the plots within the browser
%matplotlib inline 

#Importing Matplot lib and associated packages to perform Data Visualisation and Exploratory Data Analysis
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot') # optional: for ggplot-like style

# check for latest version of Matplotlib
print ('Matplotlib version: ', mpl.__version__) # >= 2.0.0

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#Importing folium to visualise Maps and plot based on Lat and Lng
import folium

#Requests to request web pages by making get requests to FourSquare REST Client
import requests

#To normalise data returned by FourSquare API
from pandas.io.json import json_normalize

#Importing KMeans from SciKit library to Classify neighborhoods into clusters
from sklearn.cluster import KMeans

print('Libraries imported')

/Users/parisring/.pyenv/versions/3.7.0/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Matplotlib version:  3.2.1
Libraries imported


In [2]:
import re 

<b> First, we start with pulling neighborhood data for LA. We scrape the data from this wikipedia site to get all of the neighborhods.</b>

In [3]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_districts_and_neighbourhoods_of_Los_Angeles').text
soup = BeautifulSoup(url,"html.parser")

In [4]:
lis = []
for li in soup.findAll('li'):
    if li.find(href="/wiki/Portal:Los_Angeles"):
        break
    if li.find(href=re.compile("^/wiki/")):
        lis.append(li)
    if li.text=='Pico Robertson[34]': #Pico Robertson is the only item on the list that does not have a hyperlink reference
        lis.append(li)

In [5]:
neigh = []
for i in range(0,len(lis)):
    neigh.append(lis[i].text.strip())
    
df = pd.DataFrame(neigh)
df.columns = ['Neighborhood']

In [6]:
df

,Neighborhood
0,Angelino Heights[1]
1,Angeles Mesa
2,Angelus Vista
3,Arleta[2][1]
4,Arlington Heights[2]
...,...
193,Wilshire Park[51]
194,Windsor Square[2][1]
195,Winnetka[2][1]
196,Woodland Hills[2][1]


<b> Now we need to reformat the dataframe, by removing any punctuation, unnecessary nmbers or duplicate records we are aware of.</b>

In [7]:
df['Neighborhood'] = df.Neighborhood.str.partition('[')[0] #Removes the citation and reference brackets
df['Neighborhood'] = df.Neighborhood.str.partition(',')[0] #Removes the alternatives for 'Bel Air'
df=df[df.Neighborhood!='Baldwin Hills/Crenshaw'] #Removes redundancy as 'Baldwin Hills' and 'Crenshaw' exist already
df=df[df.Neighborhood!='Hollywood Hills West'] #Removes redundancy as it has the same coordinates as 'Hollywood Hills'
df=df[df.Neighborhood!='Brentwood Circle'] #Removes redundancy as it has the same coordinates as 'Brentwood'
df=df[df.Neighborhood!='Wilshire Park'] #Removes redundancy as it has the same coordinates as 'Wilshire Center'
df.reset_index(inplace=True,drop=True)

In [8]:
df

,Neighborhood
0,Angelino Heights
1,Angeles Mesa
2,Angelus Vista
3,Arleta
4,Arlington Heights
...,...
189,Wilshire Center
190,Windsor Square
191,Winnetka
192,Woodland Hills


<b> We are using Nominatim to pull the coordinate data for LA neighborhoods</b>

In [9]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [10]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [11]:
column_names = ['Neighborhood', 'Latitude', 'Longitude']

nhoods = pd.DataFrame(columns=column_names)

geolocator = Nominatim(user_agent="la_explorer",timeout=5)
for i in range(0,len(df)):
    
    address = df.Neighborhood[i]+', Los Angeles'
    location = geolocator.geocode(address)
    if location == None:
        latitude = 0
        longitude = 0
    else:
        latitude = location.latitude
        longitude = location.longitude

    nhoods = nhoods.append({'Neighborhood': df.Neighborhood[i],
                                              'Latitude': latitude,
                                              'Longitude': longitude}, ignore_index=True)

In [12]:
nhoods

,Neighborhood,Latitude,Longitude
0,Angelino Heights,34.070289,-118.254796
1,Angeles Mesa,33.991402,-118.319520
2,Angelus Vista,0.000000,0.000000
3,Arleta,34.241327,-118.432205
4,Arlington Heights,34.043494,-118.321374
...,...,...,...
189,Wilshire Center,34.061515,-118.432771
190,Windsor Square,34.072593,-118.320810
191,Winnetka,34.205883,-118.570934
192,Woodland Hills,34.168436,-118.605838


<b> Then, with the neighborhood coordinates data, we reformat to floats for our coordinates, and remove any records we know would be wrong, based on the lattitude and longitudes in comparison to LA's latitude and longitude coordinates.</b>

In [13]:
nhoods['Latitude']=nhoods['Latitude'].astype(float)
nhoods['Longitude']=nhoods['Longitude'].astype(float)

nhoods=nhoods[(nhoods.Latitude>33.5) & (nhoods.Latitude<34.4) & (nhoods.Longitude<-118)] 
nhoods.reset_index(inplace=True,drop=True)

In [14]:
nhoods

,Neighborhood,Latitude,Longitude
0,Angelino Heights,34.070289,-118.254796
1,Angeles Mesa,33.991402,-118.319520
2,Arleta,34.241327,-118.432205
3,Arlington Heights,34.043494,-118.321374
4,Arts District,34.041239,-118.234450
...,...,...,...
156,Wilmington,33.780016,-118.262509
157,Wilshire Center,34.061515,-118.432771
158,Windsor Square,34.072593,-118.320810
159,Winnetka,34.205883,-118.570934


In [15]:
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.2427666.


<b> Finally, we can create a map using Folium to identify LA's neighborhoods on the map.</b>

In [16]:
map_la = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(nhoods['Latitude'], nhoods['Longitude'], nhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_la)  
    
map_la

<b> Next is connecting to Foursquare's API and pulling venue data for our Filipino Restaurants/Nearby Hotel analysis.</b>

In [17]:
CLIENT_ID = 'ZEN5NXXEEMSND55V4OGNWWRUUKATAWXAGWYQANPUYVR00VTU' # your Foursquare ID
CLIENT_SECRET = 'C4GOPGPY5DU0XEGVSKCNERQBPVLUQ1L1THGOI3GTZJA5H4JD' # your Foursquare Secret
VERSION = '20200601' # Foursquare API version
LIMIT = 1000
radius = 10000000

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: ZEN5NXXEEMSND55V4OGNWWRUUKATAWXAGWYQANPUYVR00VTU
CLIENT_SECRET:C4GOPGPY5DU0XEGVSKCNERQBPVLUQ1L1THGOI3GTZJA5H4JD


In [18]:
search_query = 'Filipino'
radius = 10000000
print(search_query + ' .... OK!')

Filipino .... OK!


In [19]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=ZEN5NXXEEMSND55V4OGNWWRUUKATAWXAGWYQANPUYVR00VTU&client_secret=C4GOPGPY5DU0XEGVSKCNERQBPVLUQ1L1THGOI3GTZJA5H4JD&ll=34.1684364,-118.6058382&v=20200601&query=Filipino&radius=10000000&limit=1000'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eed470b69babe001bf47fcb'},
 'response': {'venues': [{'id': '4bdf2a93e75c0f47da1fca03',
    'name': 'GeLourd Filipino Market',
    'location': {'address': '21534 Devonshire St',
     'lat': 34.25665855407715,
     'lng': -118.60013381666667,
     'labeledLatLngs': [{'label': 'display',
       'lat': 34.25665855407715,
       'lng': -118.60013381666667}],
     'distance': 9834,
     'postalCode': '91311',
     'cc': 'US',
     'city': 'Chatsworth',
     'state': 'CA',
     'country': 'United States',
     'formattedAddress': ['21534 Devonshire St',
      'Chatsworth, CA 91311',
      'United States']},
    'categories': [],
    'referralId': 'v-1592608656',
    'hasPerk': False},
   {'id': '4e0f572a62843e04afb486ca',
    'name': 'Filipino American Community of Los Angeles',
    'location': {'lat': 34.06891,
     'lng': -118.26436,
     'labeledLatLngs': [{'label': 'display',
       'lat': 34.06891,
       'lng': -118.26436}],
     'distance': 33363,


In [21]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/Users/parisring/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,...,location.state,location.country,location.formattedAddress,location.crossStreet,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,4bdf2a93e75c0f47da1fca03,GeLourd Filipino Market,[],v-1592608656,False,21534 Devonshire St,34.256659,-118.600134,"[{'label': 'display', 'lat': 34.25665855407715...",9834,...,CA,United States,"[21534 Devonshire St, Chatsworth, CA 91311, Un...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4e0f572a62843e04afb486ca,Filipino American Community of Los Angeles,[],v-1592608656,False,NaN,34.068910,-118.264360,"[{'label': 'display', 'lat': 34.06891, 'lng': ...",33363,...,CA,United States,"[Los Angeles, CA, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4bae6a44f964a52062af3be3,Central Filipino Seventh-Day Adventist Church,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",v-1592608656,False,777 Colorado Blvd,34.141106,-118.182614,"[{'label': 'display', 'lat': 34.14110597726068...",39105,...,CA,United States,[777 Colorado Blvd (btwn Figueroa & San Rafael...,btwn Figueroa & San Rafael,NaN,NaN,NaN,NaN,NaN,NaN
3,4c48b7f61b430f4742c993c2,Madam Camaes Filipino Palace,"[{'id': '4bf58dd8d48988d1f1931735', 'name': 'G...",v-1592608656,False,69 south bend,34.038029,-118.391256,"[{'label': 'display', 'lat': 34.03802857333641...",24534,...,CA,United States,"[69 south bend, Los Angeles, CA 90211, United ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4e392336814d9fd8faf6952d,Filipino Fusion,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",v-1592608656,False,746 Prospect Dr,34.131548,-118.245591,"[{'label': 'display', 'lat': 34.13154799999999...",33440,...,CA,United States,"[746 Prospect Dr, Glendale, CA 91205, United S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered = pd.DataFrame(dataframe_filtered)

#Remove those that are not related to filipino restaurants/eateries

dataframe_filtered = dataframe_filtered[~dataframe_filtered.categories.isin(["Church", "Martial Arts Dojo", "Car Wash", "Historic Site", "School", "Non-Profit", "None"])]
dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,GeLourd Filipino Market,None,21534 Devonshire St,34.256659,-118.600134,"[{'label': 'display', 'lat': 34.25665855407715...",9834,91311,US,Chatsworth,CA,United States,"[21534 Devonshire St, Chatsworth, CA 91311, Un...",NaN,4bdf2a93e75c0f47da1fca03
1,Filipino American Community of Los Angeles,None,NaN,34.068910,-118.264360,"[{'label': 'display', 'lat': 34.06891, 'lng': ...",33363,NaN,US,Los Angeles,CA,United States,"[Los Angeles, CA, United States]",NaN,4e0f572a62843e04afb486ca
3,Madam Camaes Filipino Palace,General Entertainment,69 south bend,34.038029,-118.391256,"[{'label': 'display', 'lat': 34.03802857333641...",24534,90211,US,Los Angeles,CA,United States,"[69 south bend, Los Angeles, CA 90211, United ...",NaN,4c48b7f61b430f4742c993c2
4,Filipino Fusion,Fried Chicken Joint,746 Prospect Dr,34.131548,-118.245591,"[{'label': 'display', 'lat': 34.13154799999999...",33440,91205,US,Glendale,CA,United States,"[746 Prospect Dr, Glendale, CA 91205, United S...",NaN,4e392336814d9fd8faf6952d
5,Filipino Bagnet Glendale Restaurant & Bakery,Bakery,1428 E Colorado St,34.142457,-118.232501,"[{'label': 'display', 'lat': 34.1424566, 'lng'...",34512,91205,US,Glendale,CA,United States,"[1428 E Colorado St, Glendale, CA 91205, Unite...",NaN,577f3126498e70c9d83fa68f
6,Historic Filipinotown,Neighborhood,NaN,34.069926,-118.275408,"[{'label': 'display', 'lat': 34.06992551754767...",32366,90026,US,Los Angeles,CA,United States,"[Los Angeles, CA 90026, United States]",NaN,4b905138f964a520b38333e3
7,LA Rsin BBQ Truck (Filipino),Food Truck,Venice Boulevard,34.094411,-118.080310,"[{'label': 'display', 'lat': 34.09441054623067...",49120,NaN,US,Los Angeles,CA,United States,"[Venice Boulevard (S. La Cienega Boulevard), L...",S. La Cienega Boulevard,568380b5498e3e34d08d0d80
8,International Express Filipino Market,Filipino Restaurant,"2413 Pacific Coast Highway, #102, Lomita, 90717",33.789787,-118.325729,"[{'label': 'display', 'lat': 33.78978729248047...",49449,90717,US,Lomita,CA,United States,"[2413 Pacific Coast Highway, #102, Lomita, 907...",NaN,4fce6655e4b0e51e2fb256f5
11,Bicol Express Filipino Fast Food,Filipino Restaurant,1220 S Central Ave,34.131464,-118.257401,"[{'label': 'display', 'lat': 34.131464, 'lng':...",32362,91204,US,Glendale,CA,United States,"[1220 S Central Ave, Glendale, CA 91204, Unite...",NaN,5587371a498edb3d83948aee
14,Pinoy Pinay Filipino Restaurant,Filipino Restaurant,11900 South St,33.857718,-118.080331,"[{'label': 'display', 'lat': 33.85771754506848...",59562,90703,US,Cerritos,CA,United States,"[11900 South St (Pioneer Ave), Cerritos, CA 90...",Pioneer Ave,4ab1b26cf964a5206f6a20e3


In [23]:
dataframe_filtered.name

0                             GeLourd Filipino Market
1          Filipino American Community of Los Angeles
3                        Madam Camaes Filipino Palace
4                                     Filipino Fusion
5        Filipino Bagnet Glendale Restaurant & Bakery
6                               Historic Filipinotown
7                        LA Rsin BBQ Truck (Filipino)
8               International Express Filipino Market
11                   Bicol Express Filipino Fast Food
14                    Pinoy Pinay Filipino Restaurant
15        Eat.Play.Move. Filipino Angeleno Food Event
17                           Lance's Filipino Cuisine
20                                 The Filipino Store
22                           Filipino Gen Merchandise
23                                 White Rabbit Truck
25                Handugan Authentic Filipino Cuisine
26                                     Hapag Filipino
27                      Filipino Bistro Asian Cuisine
28                    Lucky 

In [24]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Los Angeles',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Filipino as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [25]:
pip install geopandas 

Note: you may need to restart the kernel to use updated packages.


In [26]:
pip install rtree 

Note: you may need to restart the kernel to use updated packages.


In [27]:
import geopandas as gpd
import rtree
from shapely.geometry import Point 
print("Installed!")

Installed!


In [28]:
nhoods_geometry = gpd.GeoDataFrame(nhoods, geometry=gpd.points_from_xy(nhoods.Latitude, nhoods.Longitude))
nhoods_geometry

,Neighborhood,Latitude,Longitude,geometry
0,Angelino Heights,34.070289,-118.254796,POINT (34.070 -118.255)
1,Angeles Mesa,33.991402,-118.319520,POINT (33.991 -118.320)
2,Arleta,34.241327,-118.432205,POINT (34.241 -118.432)
3,Arlington Heights,34.043494,-118.321374,POINT (34.043 -118.321)
4,Arts District,34.041239,-118.234450,POINT (34.041 -118.234)
...,...,...,...,...
156,Wilmington,33.780016,-118.262509,POINT (33.780 -118.263)
157,Wilshire Center,34.061515,-118.432771,POINT (34.062 -118.433)
158,Windsor Square,34.072593,-118.320810,POINT (34.073 -118.321)
159,Winnetka,34.205883,-118.570934,POINT (34.206 -118.571)


In [29]:
filipino_geometry = gpd.GeoDataFrame(dataframe_filtered, geometry=gpd.points_from_xy(dataframe_filtered.lat, dataframe_filtered.lng))
filipino_geometry

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id,geometry
0,GeLourd Filipino Market,None,21534 Devonshire St,34.256659,-118.600134,"[{'label': 'display', 'lat': 34.25665855407715...",9834,91311,US,Chatsworth,CA,United States,"[21534 Devonshire St, Chatsworth, CA 91311, Un...",NaN,4bdf2a93e75c0f47da1fca03,POINT (34.257 -118.600)
1,Filipino American Community of Los Angeles,None,NaN,34.068910,-118.264360,"[{'label': 'display', 'lat': 34.06891, 'lng': ...",33363,NaN,US,Los Angeles,CA,United States,"[Los Angeles, CA, United States]",NaN,4e0f572a62843e04afb486ca,POINT (34.069 -118.264)
3,Madam Camaes Filipino Palace,General Entertainment,69 south bend,34.038029,-118.391256,"[{'label': 'display', 'lat': 34.03802857333641...",24534,90211,US,Los Angeles,CA,United States,"[69 south bend, Los Angeles, CA 90211, United ...",NaN,4c48b7f61b430f4742c993c2,POINT (34.038 -118.391)
4,Filipino Fusion,Fried Chicken Joint,746 Prospect Dr,34.131548,-118.245591,"[{'label': 'display', 'lat': 34.13154799999999...",33440,91205,US,Glendale,CA,United States,"[746 Prospect Dr, Glendale, CA 91205, United S...",NaN,4e392336814d9fd8faf6952d,POINT (34.132 -118.246)
5,Filipino Bagnet Glendale Restaurant & Bakery,Bakery,1428 E Colorado St,34.142457,-118.232501,"[{'label': 'display', 'lat': 34.1424566, 'lng'...",34512,91205,US,Glendale,CA,United States,"[1428 E Colorado St, Glendale, CA 91205, Unite...",NaN,577f3126498e70c9d83fa68f,POINT (34.142 -118.233)
6,Historic Filipinotown,Neighborhood,NaN,34.069926,-118.275408,"[{'label': 'display', 'lat': 34.06992551754767...",32366,90026,US,Los Angeles,CA,United States,"[Los Angeles, CA 90026, United States]",NaN,4b905138f964a520b38333e3,POINT (34.070 -118.275)
7,LA Rsin BBQ Truck (Filipino),Food Truck,Venice Boulevard,34.094411,-118.080310,"[{'label': 'display', 'lat': 34.09441054623067...",49120,NaN,US,Los Angeles,CA,United States,"[Venice Boulevard (S. La Cienega Boulevard), L...",S. La Cienega Boulevard,568380b5498e3e34d08d0d80,POINT (34.094 -118.080)
8,International Express Filipino Market,Filipino Restaurant,"2413 Pacific Coast Highway, #102, Lomita, 90717",33.789787,-118.325729,"[{'label': 'display', 'lat': 33.78978729248047...",49449,90717,US,Lomita,CA,United States,"[2413 Pacific Coast Highway, #102, Lomita, 907...",NaN,4fce6655e4b0e51e2fb256f5,POINT (33.790 -118.326)
11,Bicol Express Filipino Fast Food,Filipino Restaurant,1220 S Central Ave,34.131464,-118.257401,"[{'label': 'display', 'lat': 34.131464, 'lng':...",32362,91204,US,Glendale,CA,United States,"[1220 S Central Ave, Glendale, CA 91204, Unite...",NaN,5587371a498edb3d83948aee,POINT (34.131 -118.257)
14,Pinoy Pinay Filipino Restaurant,Filipino Restaurant,11900 South St,33.857718,-118.080331,"[{'label': 'display', 'lat': 33.85771754506848...",59562,90703,US,Cerritos,CA,United States,"[11900 South St (Pioneer Ave), Cerritos, CA 90...",Pioneer Ave,4ab1b26cf964a5206f6a20e3,POINT (33.858 -118.080)


In [40]:
filipino_pts_clean = filipino_geometry[filipino_geometry.geometry.type == 'Point']
nhoods_pts_clean = nhoods_geometry[nhoods_geometry.geometry.type == 'Point']
test_f_join = gpd.sjoin(nhoods_pts_clean, filipino_pts_clean, op='intersects')
test_f_join.head()

,Neighborhood,Latitude,Longitude,geometry,index_right,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
